In [1]:
using Mocha

Configuring Mocha...
 * CUDA       disabled by default
 * Native Ext disabled by default
Mocha configured, continue loading module...
DefaultBackend = Mocha.CPUBackend


In [2]:
train_filename = "cc_train.txt"
test_filename = "cc_test.txt"
exp_dir = "snapshots/cc_net_2/"

"snapshots/cc_net_2/"

In [3]:
data_layer  = HDF5DataLayer(name="train-data", source="cc_data/txt/$train_filename",
batch_size=64, shuffle=true)

Mocha.HDF5DataLayer(train-data)

In [4]:
fc1_layer  = InnerProductLayer(name="ip1", output_dim=256,
neuron=Neurons.ReLU(), bottoms=[:data], tops=[:ip1])
fc2_layer  = InnerProductLayer(name="ip2", output_dim=256,
neuron=Neurons.ReLU(), bottoms=[:ip1], tops=[:ip2])

fc3_layer  = InnerProductLayer(name="ip3", output_dim=64,
neuron=Neurons.ReLU(), bottoms=[:ip2], tops=[:ip3])

fc4_layer  = InnerProductLayer(name="ip4", output_dim=64,
neuron=Neurons.Sigmoid(), bottoms=[:ip3], tops=[:ip4])

fc5_layer  = InnerProductLayer(name="ip5", output_dim=32,
neuron=Neurons.Sigmoid(), bottoms=[:ip4], tops=[:ip5])

Mocha.InnerProductLayer(ip5)

In [5]:
fc6_layer  = InnerProductLayer(name="ip6", output_dim=2,
bottoms=[:ip5], tops=[:ip6])

Mocha.InnerProductLayer(ip6)

In [6]:
# drop_input  = DropoutLayer(name="drop_in", bottoms=[:data], ratio=0.2)
# drop_fc1 = DropoutLayer(name="drop_fc1", bottoms=[:ip1], ratio=0.5)
# drop_fc2 = DropoutLayer(name="drop_fc2", bottoms=[:ip2], ratio=0.5)
# drop_fc3 = DropoutLayer(name="drop_fc3", bottoms=[:ip3], ratio=0.5)
# drop_fc4 = DropoutLayer(name="drop_fc4", bottoms=[:ip4], ratio=0.5)

In [7]:
loss_layer = SoftmaxLossLayer(name="loss", bottoms=[:ip6,:label])

Mocha.SoftmaxLossLayer(loss)

In [8]:
backend = CPUBackend()
init(backend)

In [9]:
common_layers = [fc1_layer, fc2_layer, fc3_layer, fc4_layer, fc5_layer, fc6_layer]
# drop_layers = [drop_input, drop_fc1, drop_fc2, drop_fc3, drop_fc4] ;

In [10]:
net = Net("cc-train", backend, [data_layer, common_layers..., loss_layer]) ;

[2018-10-14T22:02:12 | info | Mocha]: Constructing net cc-train on Mocha.CPUBackend...
[2018-10-14T22:02:12 | info | Mocha]: Topological sorting 8 layers...
[2018-10-14T22:02:12 | info | Mocha]: Setup layers...
[2018-10-14T22:02:15 | info | Mocha]: Network constructed!


In [11]:
method = SGD()
params = make_solver_parameters(method, max_iter=10000, regu_coef=0.0005,
    mom_policy=MomPolicy.Fixed(0.9),
    lr_policy=LRPolicy.Inv(0.03, 0.0001, 0.75),
    load_from=exp_dir)

Dict{Symbol,Any} with 5 entries:
  :lr_policy  => Mocha.LRPolicy.Inv(0.03, 0.0001, 0.75)
  :max_iter   => 10000
  :load_from  => "snapshots/cc_net_2/"
  :regu_coef  => 0.0005
  :mom_policy => Mocha.MomPolicy.Fixed(0.9)

In [12]:
solver = Solver(method, params)

Mocha.Solver{Mocha.SGD}(Mocha.SGD(), Dict{Symbol,Any}(Pair{Symbol,Any}(:lr_policy, Mocha.LRPolicy.Inv(0.03, 0.0001, 0.75)),Pair{Symbol,Any}(:max_iter, 10000),Pair{Symbol,Any}(:load_from, "snapshots/cc_net_2/"),Pair{Symbol,Any}(:regu_coef, 0.0005),Pair{Symbol,Any}(:mom_policy, Mocha.MomPolicy.Fixed(0.9))), Mocha.CoffeeLounge("", 1, :merge, Dict{AbstractString,Dict{Int64,AbstractFloat}}(), Mocha.CoffeeBreak[], false, 0, 0))

In [13]:
setup_coffee_lounge(solver, save_into="$exp_dir/statistics.jld", every_n_iter=1000)

:merge

In [14]:
add_coffee_break(solver, TrainingSummary(), every_n_iter=100)
add_coffee_break(solver, Snapshot(exp_dir), every_n_iter=500)

2-element Array{Mocha.CoffeeBreak,1}:
 Mocha.CoffeeBreak(Mocha.TrainingSummary(Any[:iter, :obj_val]), 100, 0)
 Mocha.CoffeeBreak(Mocha.Snapshot("snapshots/cc_net_2/"), 500, 0)      

In [15]:
data_layer_test = HDF5DataLayer(name="test-data", source="cc_data/txt/$test_filename", batch_size=100)
acc_layer = AccuracyLayer(name="test-accuracy", bottoms=[:ip6, :label])
test_net = Net("cc-test", backend, [data_layer_test, common_layers..., acc_layer]) ;

[2018-10-14T22:02:18 | info | Mocha]: Constructing net cc-test on Mocha.CPUBackend...
[2018-10-14T22:02:18 | info | Mocha]: Topological sorting 8 layers...
[2018-10-14T22:02:18 | info | Mocha]: Setup layers...
[2018-10-14T22:02:18 | info | Mocha]: Network constructed!


In [16]:
add_coffee_break(solver, ValidationPerformance(test_net), every_n_iter=1000);

In [17]:
@time solve(solver, net)

destroy(net)
destroy(test_net)
shutdown(backend)

[2018-10-14T22:02:29 | info | Mocha]: Snapshot directory snapshots/cc_net_2/ already exists
[2018-10-14T22:02:29 | info | Mocha]:  TRAIN iter=000000 obj_val=0.70940685
[2018-10-14T22:02:30 | info | Mocha]: Saving snapshot to snapshot-000000.jld...
[2018-10-14T22:02:32 | info | Mocha]: 
[2018-10-14T22:02:32 | info | Mocha]: ## Performance on Validation Set after 0 iterations
[2018-10-14T22:02:32 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:02:33 | info | Mocha]:   Accuracy (avg over 6000) = 27.1167%
[2018-10-14T22:02:33 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:02:33 | info | Mocha]: 
[2018-10-14T22:02:35 | info | Mocha]:  TRAIN iter=000100 obj_val=0.64727199
[2018-10-14T22:02:35 | info | Mocha]:  TRAIN iter=000200 obj_val=0.49666008
[2018-10-14T22:02:35 | info | Mocha]:  TRAIN iter=000300 obj_val=0.52714467
[2018-10-14T22:02:36 | info | Mocha]:  TRAIN iter=000400 obj_val=0.46459329
[2018-10-14T

[2018-10-14T22:02:50 | info | Mocha]:  TRAIN iter=006000 obj_val=0.47732368
[2018-10-14T22:02:50 | info | Mocha]: Saving snapshot to snapshot-006000.jld...
[2018-10-14T22:02:50 | info | Mocha]: 
[2018-10-14T22:02:50 | info | Mocha]: ## Performance on Validation Set after 6000 iterations
[2018-10-14T22:02:50 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:02:50 | info | Mocha]:   Accuracy (avg over 6000) = 78.1333%
[2018-10-14T22:02:50 | info | Mocha]: ---------------------------------------------------------
[2018-10-14T22:02:50 | info | Mocha]: 
[2018-10-14T22:02:51 | info | Mocha]:  TRAIN iter=006100 obj_val=0.60060340
[2018-10-14T22:02:51 | info | Mocha]:  TRAIN iter=006200 obj_val=0.52636528
[2018-10-14T22:02:51 | info | Mocha]:  TRAIN iter=006300 obj_val=0.54498464
[2018-10-14T22:02:52 | info | Mocha]:  TRAIN iter=006400 obj_val=0.50471437
[2018-10-14T22:02:52 | info | Mocha]:  TRAIN iter=006500 obj_val=0.48471397
[2018-10-14T22:02:52 | in

Dict{AbstractString,Array{Mocha.AbstractParameter,1}} with 0 entries